In [1]:
!pip install flask-ngrok
!pip install torch torchvision
!pip install transformers
!pip install Pillow
!pip install numpy
!pip install opencv-python-headless
!pip install pytesseract


  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [2]:
from flask import Flask, request, jsonify, render_template, send_from_directory
from flask_ngrok import run_with_ngrok
from PIL import Image
import numpy as np
import os
import torch
from transformers import CLIPProcessor, CLIPModel
import cv2
import pytesseract

# Initialize Flask app
app = Flask(__name__)
run_with_ngrok(app)  # Starts ngrok when the app is run

# Initialize the CLIP model and processor
clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

# Create directories if not exist
if not os.path.exists('static/output'):
    os.makedirs('static/output')

@app.route('/')
def home():
    return '''
        <!DOCTYPE html>
        <html lang="en">
        <head>
            <meta charset="UTF-8">
            <meta name="viewport" content="width=device-width, initial-scale=1.0">
            <title>AI Image Segmentation</title>
        </head>
        <body>
            <h1>Upload an Image for Segmentation</h1>
            <form action="/process" method="post" enctype="multipart/form-data">
                <input type="file" name="file" accept="image/*">
                <button type="submit">Upload and Process</button>
            </form>
        </body>
        </html>
    '''

@app.route('/process', methods=['POST'])
def process():
    if 'file' not in request.files:
        return "No file part", 400
    file = request.files['file']
    if file.filename == '':
        return "No selected file", 400

    # Save the uploaded file
    input_path = os.path.join('static/output', file.filename)
    file.save(input_path)

    # Process the image (dummy segmentation and extraction for illustration)
    image = Image.open(input_path)
    image_np = np.array(image)

    # Dummy segmentation (you should replace this with your actual segmentation code)
    segmented_image = np.zeros_like(image_np)
    segmented_image[50:150, 50:150] = image_np[50:150, 50:150]

    # Save segmented image
    segmented_path = os.path.join('static/output', 'segmented_' + file.filename)
    Image.fromarray(segmented_image).save(segmented_path)

    # Dummy object description
    inputs = clip_processor(images=image, return_tensors="pt")
    outputs = clip_model(**inputs)
    description = "Object detected"

    # Extract text (dummy example)
    extracted_text = pytesseract.image_to_string(image)

    # Prepare result
    result = {
        "description": description,
        "extracted_text": extracted_text,
        "segmented_image_url": segmented_path
    }

    return jsonify(result)

@app.route('/static/output/<path:filename>')
def download_file(filename):
    return send_from_directory('static/output', filename)

if __name__ == '__main__':
    app.run()


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
Exception in thread Thread-11:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/urllib3/connection.py", line 203, in _new_conn
    sock = connection.create_connection(
  File "/usr/local/lib/python3.10/dist-packages/urllib3/util/connection.py", line 85, in create_connection
    raise err
  File "/usr/local/lib/python3.10/dist-packages/urllib3/util/connection.py", line 73, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [Errno 111] Connection refused

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py", line 791, in urlopen
    response = self._make_request(
  File "/usr/local/lib/python3.10/dist-packages/urll

In [3]:
from google.colab import files
uploaded = files.upload()

Saving set-of-random-objects.jpg to set-of-random-objects.jpg


In [ ]:
from flask import Flask, request, jsonify, render_template, send_from_directory
from flask_ngrok import run_with_ngrok
from PIL import Image
import numpy as np
import os
import torch
from transformers import CLIPProcessor, CLIPModel
import cv2
import pytesseract

app = Flask(__name__)
run_with_ngrok(app)  # Start ngrok when app is run

# Initialize the CLIP model and processor
clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

@app.route('/')
def home():
    return '''
        <!DOCTYPE html>
        <html lang="en">
        <head>
            <meta charset="UTF-8">
            <meta name="viewport" content="width=device-width, initial-scale=1.0">
            <title>AI Image Segmentation</title>
        </head>
        <body>
            <h1>Upload an Image for Segmentation</h1>
            <form action="/process" method="post" enctype="multipart/form-data">
                <input type="file" name="file" accept="image/*">
                <button type="submit">Upload and Process</button>
            </form>
        </body>
        </html>
    '''

@app.route('/process', methods=['POST'])
def process():
    if 'file' not in request.files:
        return "No file part", 400
    file = request.files['file']
    if file.filename == '':
        return "No selected file", 400

    # Save the uploaded file
    input_path = os.path.join('static/output', file.filename)
    file.save(input_path)

    # Process the image (dummy segmentation and extraction for illustration)
    image = Image.open('set-of-random-objects.jpg')
    image_np = np.array(image)

    # Dummy segmentation (you should replace this with your actual segmentation code)
    segmented_image = np.zeros_like(image_np)
    segmented_image[50:150, 50:150] = image_np[50:150, 50:150]

    # Save segmented image
    segmented_path = os.path.join('static/output', 'segmented_' + file.filename)
    Image.fromarray(segmented_image).save(segmented_path)

    # Dummy object description
    inputs = clip_processor(images=image, return_tensors="pt")
    outputs = clip_model(**inputs)
    description = "Object detected"

    # Extract text (dummy example)
    extracted_text = pytesseract.image_to_string(image)

    # Prepare result
    result = {
        "description": description,
        "extracted_text": extracted_text,
        "segmented_image_url": segmented_path
    }

    return jsonify(result)

@app.route('/static/output/<path:filename>')
def download_file(filename):
    return send_from_directory('static/output', filename)

if __name__ == '__main__':
    app.run()


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
Exception in thread Thread-12:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/urllib3/connection.py", line 203, in _new_conn
    sock = connection.create_connection(
  File "/usr/local/lib/python3.10/dist-packages/urllib3/util/connection.py", line 85, in create_connection
    raise err
  File "/usr/local/lib/python3.10/dist-packages/urllib3/util/connection.py", line 73, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [Errno 111] Connection refused

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py", line 791, in urlopen
    response = self._make_request(
  File "/usr/local/lib/python3.10/dist-packages/urll